In [1]:
import statsmodels

import pandas as pd
import statsmodels.formula.api as smf

In [2]:
print(pd.__version__)
print(statsmodels.__version__)

0.25.1
0.11.1


Different versions of statsmodels may give different results

# Why to use method='ncg', 101, unless you like unnecessary warnings

In [3]:
df = pd.DataFrame.from_dict({
    's' : [0] * 50 + [1] * 50,
    'a' : [0] * 50 + [1] * 50
})

In [4]:
%time res = smf.logit('s ~ a - 1', data=df).fit()

         Current function value: 0.346574
         Iterations: 35
CPU times: user 18.1 ms, sys: 3.69 ms, total: 21.8 ms
Wall time: 19.4 ms


/usr/local/lib/python3.7/site-packages/statsmodels/base/model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


In [5]:
print(res.summary2())

                        Results: Logit
Model:               Logit            Pseudo R-squared: 0.500  
Dependent Variable:  s                AIC:              71.3147
Date:                2020-08-05 20:20 BIC:              73.9199
No. Observations:    100              Log-Likelihood:   -34.657
Df Model:            0                LL-Null:          -69.315
Df Residuals:        99               LLR p-value:      nan    
Converged:           0.0000           Scale:            1.0000 
No. Iterations:      35.0000                                   
---------------------------------------------------------------
   Coef.    Std.Err.     z    P>|z|      [0.025       0.975]   
---------------------------------------------------------------
a 31.7877 1126330.1529 0.0000 1.0000 -2207534.7467 2207598.3221



In [6]:
%time res = smf.logit('s ~ a - 1', data=df).fit(method='ncg')

Optimization terminated successfully.
         Current function value: 0.346580
         Iterations: 11
         Function evaluations: 12
         Gradient evaluations: 22
         Hessian evaluations: 11
CPU times: user 12.5 ms, sys: 2.05 ms, total: 14.6 ms
Wall time: 12.7 ms


In [7]:
print(res.summary2())

                        Results: Logit
Model:              Logit            Pseudo R-squared: 0.500  
Dependent Variable: s                AIC:              71.3161
Date:               2020-08-05 20:20 BIC:              73.9213
No. Observations:   100              Log-Likelihood:   -34.658
Df Model:           0                LL-Null:          -69.315
Df Residuals:       99               LLR p-value:      nan    
Converged:          1.0000           Scale:            1.0000 
----------------------------------------------------------------
        Coef.    Std.Err.     z      P>|z|     [0.025     0.975]
----------------------------------------------------------------
a      11.2029    38.2996   0.2925   0.7699   -63.8629   86.2687



There isn't much difference between 31.7877 and 11.2029 given how sigmoid function works.

# Why to use method='ncg', 102, unless you like erros that prevent you from checking the results and make your own conclusions about what is going on

In [8]:
df = pd.DataFrame.from_dict({
    's' : [0] * 50 + [1] * 50,
    'a' : [0] * 100
})

In [9]:
%time res = smf.logit('s ~ a - 1', data=df).fit()

Optimization terminated successfully.
         Current function value: 0.693147
         Iterations 1


LinAlgError: Singular matrix

In [10]:
print(res.summary2())

                        Results: Logit
Model:              Logit            Pseudo R-squared: 0.500  
Dependent Variable: s                AIC:              71.3161
Date:               2020-08-05 20:20 BIC:              73.9213
No. Observations:   100              Log-Likelihood:   -34.658
Df Model:           0                LL-Null:          -69.315
Df Residuals:       99               LLR p-value:      nan    
Converged:          1.0000           Scale:            1.0000 
----------------------------------------------------------------
        Coef.    Std.Err.     z      P>|z|     [0.025     0.975]
----------------------------------------------------------------
a      11.2029    38.2996   0.2925   0.7699   -63.8629   86.2687



In [11]:
%time res = smf.logit('s ~ a - 1', data=df).fit(method='ncg')

Optimization terminated successfully.
         Current function value: 0.693147
         Iterations: 1
         Function evaluations: 2
         Gradient evaluations: 2
         Hessian evaluations: 1
CPU times: user 8.79 ms, sys: 505 µs, total: 9.3 ms
Wall time: 8.88 ms


/usr/local/lib/python3.7/site-packages/statsmodels/base/model.py:548: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInversionWarning)


In [12]:
print(res.summary2())

                        Results: Logit
Model:              Logit            Pseudo R-squared: 0.000   
Dependent Variable: s                AIC:              138.6294
Date:               2020-08-05 20:20 BIC:              138.6294
No. Observations:   100              Log-Likelihood:   -69.315 
Df Model:           -1               LL-Null:          -69.315 
Df Residuals:       100              LLR p-value:      nan     
Converged:          1.0000           Scale:            1.0000  
-------------------------------------------------------------------
        Coef.      Std.Err.      z      P>|z|     [0.025     0.975]
-------------------------------------------------------------------
a       0.0000          nan     nan       nan        nan        nan



Seeing the results help me understand what is going on, but LinAlgError? Oh hell I don't want to read the source code to find out that it is due to failed hessian inversion.

## A more elaborate example

In [13]:
df = pd.DataFrame.from_dict({
    's' : [0] * 49 + [1] * 1 + [0] * 49 + [1] * 1 + [1] * 49 + [0] * 1,
    'a' : [0] * 50           + [1] * 50           + [1] * 50,
    'b' : [0] * 50           + [0] * 50           + [1] * 50
})

In [14]:
%time res = smf.logit('s ~ C(a) : C(b) - 1', data=df).fit()

Optimization terminated successfully.
         Current function value: 0.098039
         Iterations 8


LinAlgError: Singular matrix

In [15]:
%time res = smf.logit('s ~ C(a) : C(b) - 1', data=df).fit(method='ncg')

Optimization terminated successfully.
         Current function value: 0.098039
         Iterations: 7
         Function evaluations: 8
         Gradient evaluations: 14
         Hessian evaluations: 7
CPU times: user 14 ms, sys: 2.59 ms, total: 16.6 ms
Wall time: 14.4 ms


/usr/local/lib/python3.7/site-packages/statsmodels/base/model.py:548: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInversionWarning)


In [16]:
print(res.summary2())

                         Results: Logit
Model:              Logit            Pseudo R-squared: 0.847     
Dependent Variable: s                AIC:              35.4117   
Date:               2020-08-05 20:20 BIC:              44.4436   
No. Observations:   150              Log-Likelihood:   -14.706   
Df Model:           2                LL-Null:          -96.155   
Df Residuals:       147              LLR p-value:      4.2360e-36
Converged:          1.0000           Scale:            1.0000    
------------------------------------------------------------------
                      Coef.   Std.Err.   z   P>|z|  [0.025  0.975]
------------------------------------------------------------------
C(a)[0]:C(b)[0]      -3.8918       nan  nan    nan     nan     nan
C(a)[1]:C(b)[0]      -3.8918       nan  nan    nan     nan     nan
C(a)[0]:C(b)[1]       0.0000       nan  nan    nan     nan     nan
C(a)[1]:C(b)[1]       3.8918       nan  nan    nan     nan     nan



/usr/local/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
/usr/local/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
/usr/local/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:1912: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)


Seeing C(a)[0]:C(b)[1] has coefficient 0.0000 helps to determine that there is some missing combinations in the data.

Let's try to fix this

In [17]:
df

,s,a,b
0,0,0,0
1,0,0,0
2,0,0,0
3,0,0,0
4,0,0,0
...,...,...,...
145,1,1,1
146,1,1,1
147,1,1,1
148,1,1,1


In [18]:
df['a_b'] = df.a.apply(str) + '_' + df.b.apply(str)

In [19]:
df

,s,a,b,a_b
0,0,0,0,0_0
1,0,0,0,0_0
2,0,0,0,0_0
3,0,0,0,0_0
4,0,0,0,0_0
...,...,...,...,...
145,1,1,1,1_1
146,1,1,1,1_1
147,1,1,1,1_1
148,1,1,1,1_1


In [20]:
%time res = smf.logit('s ~ C(a_b) - 1', data=df).fit()

Optimization terminated successfully.
         Current function value: 0.098039
         Iterations 8
CPU times: user 16.6 ms, sys: 4.31 ms, total: 20.9 ms
Wall time: 11.9 ms


In [21]:
print(res.summary2())

                         Results: Logit
Model:              Logit            Pseudo R-squared: 0.847     
Dependent Variable: s                AIC:              35.4117   
Date:               2020-08-05 20:20 BIC:              44.4436   
No. Observations:   150              Log-Likelihood:   -14.706   
Df Model:           2                LL-Null:          -96.155   
Df Residuals:       147              LLR p-value:      4.2360e-36
Converged:          1.0000           Scale:            1.0000    
No. Iterations:     8.0000                                       
------------------------------------------------------------------
               Coef.   Std.Err.     z     P>|z|    [0.025   0.975]
------------------------------------------------------------------
C(a_b)[0_0]   -3.8918    1.0102  -3.8527  0.0001  -5.8717  -1.9120
C(a_b)[1_0]   -3.8918    1.0102  -3.8527  0.0001  -5.8717  -1.9120
C(a_b)[1_1]    3.8918    1.0102   3.8527  0.0001   1.9120   5.8717



In [22]:
%time res = smf.logit('s ~ C(a_b) - 1', data=df).fit(method='ncg')

Optimization terminated successfully.
         Current function value: 0.098039
         Iterations: 7
         Function evaluations: 8
         Gradient evaluations: 14
         Hessian evaluations: 7
CPU times: user 11.7 ms, sys: 2 ms, total: 13.7 ms
Wall time: 12.1 ms


In [23]:
print(res.summary2())

                         Results: Logit
Model:              Logit            Pseudo R-squared: 0.847     
Dependent Variable: s                AIC:              35.4117   
Date:               2020-08-05 20:20 BIC:              44.4436   
No. Observations:   150              Log-Likelihood:   -14.706   
Df Model:           2                LL-Null:          -96.155   
Df Residuals:       147              LLR p-value:      4.2360e-36
Converged:          1.0000           Scale:            1.0000    
------------------------------------------------------------------
               Coef.   Std.Err.     z     P>|z|    [0.025   0.975]
------------------------------------------------------------------
C(a_b)[0_0]   -3.8918    1.0102  -3.8527  0.0001  -5.8717  -1.9120
C(a_b)[1_0]   -3.8918    1.0102  -3.8527  0.0001  -5.8717  -1.9120
C(a_b)[1_1]    3.8918    1.0102   3.8527  0.0001   1.9120   5.8717

